In [1]:
import os
from openpyxl import Workbook
from openpyxl import load_workbook as lw
from openpyxl.styles import Color, PatternFill, Style
import pandas as pd

In [2]:
vwb = lw(filename='./../database_files/Escherichia_VFs_comparsion.xlsx', read_only = True)
vs = vwb['VFs_comparison']

In [3]:
#phyl is a list of the things in the phylogentic tree order
with open('./../Phylo_tree_order.txt', 'r') as PO:
    phyo = PO.read()
PO.close
phyl = phyo.split('\n')

In [4]:
vglist = []
for i in range(4, vs.max_row):
    vg = None
    vg = vs['B'+str(i)].value
    if vg == None:
        vg = vs['A'+str(i)].value
    else:
        pass
    vglist.append(vg)


In [6]:
obw = lw(filename = 'VFDB_PRO_BLAST_TABLE.xlsx', read_only = True)
obs = obw['In Text Figure']

In [7]:
#We've already build indices for the chart. Now let's build the table with that information
#This provides an easy way to find the location of a gene in the blast results worksheet
bscols = []
for i in range(2, obs.max_column+1):
    bscols.append(obs.cell(row = 1, column = i).value)
bsrows = []
for i in range(2, obs.max_row+1):
    bsrows.append(obs.cell(row = i, column = 1).value)
    print(i)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89


In [8]:
#build a pandas database from the bs database

bw = pd.ExcelFile('VFDB_PRO_BLAST_TABLE.xlsx')
bs = bw.parse('In Text Figure')

In [9]:
bs.head()
print(bs.get_value('S01', 'espG'))
#BOOYAKASHA!

1


In [10]:
wb = Workbook()
ws = wb.active
ws.name = ['PosterChart']
#build row 1
fillyes = PatternFill(fill_type='solid', start_color='0000FF', end_color='0000FF')
fillno = PatternFill(fill_type='solid', start_color='CCCCCC', end_color='CCCCCC')

ws['A2'].value = 'FDA Name'

col = 2
ro = 3
vgcol = []
for item in vglist:
    if item[0].isupper():
        ws.cell(column = col, row = 1).value = item
    elif '/' in item:
        continue
    elif item not in bscols:
        continue
    else:
        ws.cell(column = col, row = 2).value = item
        col += 1
        vgcol.append(item)

for item in phyl:
    ws.cell(column = 1, row = ro).value = item
    for jtem in vgcol:
        if '/' in jtem:
            continue
        if jtem[0].islower() and jtem in bscols:
            bsc = bscols.index(jtem)+2
            bsr = bsrows.index(item)+2
            info = float(bs.get_value(item, jtem))
            ws.cell(column = vgcol.index(jtem)+2, row = ro).value = info
            if float(info) >= 0.6:
                ws.cell(column = vgcol.index(jtem)+2, row = ro).fill = fillyes
            else:
                ws.cell(column = vgcol.index(jtem)+2, row = ro).fill = fillno
        else:
            continue
    ro+=1
wb.save(filename = 'Polished_table_pro_wS17.xlsx')
                